In [64]:
import json
import os
from json import JSONDecodeError
import sys
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go

In [65]:
os.listdir()

['DR_FSP_DR_PFSP',
 'DR_SP_DR_FSP',
 'DR_SP_DR_PFSP',
 'parse_results',
 'rr_parsing.py',
 'rr_viz.ipynb']

In [66]:
def plot_returns(values: pd.Series, agent_name: str):
    df = pd.DataFrame(values)
    df.columns = ["values"]
    df["avg"] = df["values"].apply(lambda x: np.array(x).mean())
    df["std"] = df["values"].apply(lambda x: np.array(x).std())

    df["lower"] = df["avg"] - df["std"]
    df["upper"] = df["avg"] + df["std"]

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=np.array([df.index, df.index[::-1]]).flatten(),
            y=np.array([df["upper"], df["lower"][::-1]]).flatten(),
            fill="toself",
            fillcolor="rgba(0, 0, 255, 0.2)",
            line=dict(color="rgba(255, 255, 255, 0)"),
            name="Error Band",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df["avg"],
            mode="lines",
            line=dict(color="blue"),
            name="Average",
        )
    )

    fig.update_layout(
        title=agent_name,
        xaxis_title="Checkpoints",
        yaxis_title="Returns",
        showlegend=True,
    )
    fig.show()

In [67]:
def plot_logs(agent_1: str, agent_2: str):
    with open(f"parse_results/{agent_1}_{agent_2}_returns.json", "r") as f:
        logs = json.load(f)
    plot_returns(pd.Series(logs[agent_1]), agent_1)
    plot_returns(pd.Series(logs[agent_2]), agent_2)

In [68]:
plot_logs("DR_SP", "DR_FSP")

In [69]:
plot_logs("DR_SP", "DR_PFSP")

In [70]:
plot_logs("DR_FSP", "DR_PFSP")

In [71]:
def plot_logs(agent_1: str, agent_2: str):
    with open(f"parse_results/{agent_1}_{agent_2}_returns.json", "r") as f:
        logs = json.load(f)
    plot_returns(pd.Series(logs[agent_1]), agent_1)
    plot_returns(pd.Series(logs[agent_2]), agent_2)

In [73]:
df = {}
logs = [file for file in os.listdir("parse_results") if ".json" in file]
for log in logs:
    print(log) 
    with open(f"parse_results/{log}", "r") as f:
        returns = json.load(f)

    df[log] = returns

DR_FSP_DR_PFSP_returns.json
DR_SP_DR_FSP_returns.json
DR_SP_DR_PFSP_returns.json


In [74]:
for log in list(df.keys()):
    for agent in df[log].keys():
        print(agent)
        print(len(list(df[log][agent].keys())))

DR_FSP
109
DR_PFSP
109
DR_SP
40
DR_FSP
40
DR_SP
40
DR_PFSP
40


In [95]:
df = {}
logs = [file for file in os.listdir("parse_results") if ".json" in file]
for log in logs:
    print(log) 
    with open(f"parse_results/{log}", "r") as f:
        returns = json.load(f)

        agent_1, agent_2 = list(returns.keys())
        if agent_1 not in df.keys():
            df[agent_1] = returns[agent_1]
        if agent_2 not in df.keys():
            df[agent_2] = returns[agent_2]
        else:
            for checkpoint in list(returns[agent_1].keys()):
                df[agent_1][checkpoint].extend(returns[agent_1][checkpoint])
            for checkpoint in list(returns[agent_2].keys()):
                df[agent_2][checkpoint].extend(returns[agent_2][checkpoint])

df = pd.DataFrame(df)

df.index = np.array(df.index).astype(int)
df = df.sort_index()
# df = df.head(40)

agents = df.columns
fig = go.Figure()

for agent in agents:
    df[f"{agent}_avg"] = df[agent].apply(lambda x: np.array(x).mean())
    df[f"{agent}_std"] = df[agent].apply(lambda x: np.array(x).std())

    df[f"{agent}_lower"] = df[f"{agent}_avg"] - df[f"{agent}_std"]
    df[f"{agent}_upper"] = df[f"{agent}_avg"] + df[f"{agent}_std"]


    # fig.add_trace(
    #     go.Scatter(
    #         x=np.array([df.index, df.index[::-1]]).flatten(),
    #         y=np.array([df[f"{agent}_upper"], df[f"{agent}_lower"][::-1]]).flatten(),
    #         fill="toself",
    #         fillcolor="rgba(0, 0, 255, 0.2)",
    #         line=dict(color="rgba(255, 255, 255, 0)"),
    #         name="Error Band",
    #     )
    # )
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[f"{agent}_avg"],
            mode="lines",
            name=agent,
        )
    )

fig.update_layout(
    title="RR returns",
    xaxis_title="Checkpoints",
    yaxis_title="Returns",
    showlegend=True,
)
fig.show()

DR_FSP_DR_PFSP_returns.json
DR_SP_DR_FSP_returns.json
DR_SP_DR_PFSP_returns.json


In [88]:
df["DR_SP"].dropna().shape

(40,)